### Задание 1
Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. 
Чем они отличаются и чем они похожи?  
Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?

User-User - объект понравится пользователю, если он понравился похожим пользователям.  
Item-Item - объект понравится пользователю, если ему понравились похожие объекты  

1. Сходство:
    1. в обоих случаях пытаемся понять понравится ли товар пользователю  
1. Различие 
    1. В первом случае говорим что объект понравиться если 
       этот объект понравился пользователям похожим того для которого производится поиск, а 
       во втором случае говорим, что понравится если пользователю понравился товар похожий на тот для которого производится поиск.
    1. У User-User обычно работает дольше из-за того что пользователей обычно больше чем товаров
    1. Item-Item более точный потому что пользоватей обычно больше чем товаров
    
Не понял вопросы "Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?" 

### Задание 2
В чем принципиальные отличия item-item рекомендаций от ALS?  
Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров

item-item - это "направление" поиска, а ALS алгоритм поиска.

Примеры весов:
1. Лайки/дизлайки (реакции)  
1. Коэффициент цитируемости 
1. Количество просмотров/прослушиваний и т.д.

### Задание 3
Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

Не понял вопроса

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [3]:
def id_converter(C):
    userids = C.index.values
    itemids = C.columns.values

    matrix_u_ids = np.arange(len(userids))
    matrix_i_ids = np.arange(len(itemids))

    id_to_itemid = dict(zip(matrix_i_ids, itemids))
    id_to_userid = dict(zip(matrix_u_ids, userids))

    itemid_to_id = dict(zip(itemids, matrix_i_ids))
    userid_to_id = dict(zip(userids, matrix_u_ids))
    
    return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

In [4]:
def get_recommends(user, user_item, filter_items, model,  N=5):
    ids, scores = model.recommend(userid=user, 
                                  user_items=user_item,   # на вход user-item matrix
                                  N=N, 
                                  filter_already_liked_items=False, 
                                  filter_items=filter_items, 
                                  recalculate_user=True)
    return ids

In [5]:
def als(data, values, arg_func, u_ids, filter_item, model_params):
    user_item_matrix = pd.pivot_table(data, index='user_id', columns='item_id', 
                                      values=values, aggfunc=arg_func, fill_value=0)
    user_item_matrix = user_item_matrix.astype(float)
    sparse_uim = csr_matrix(user_item_matrix).tocsr()
    id_to_itemid, id_to_userid, itemid_to_id, userid_to_id = id_converter(user_item_matrix)
    
    model = AlternatingLeastSquares(factors=model_params['factors'], 
                                    regularization=model_params['regularization'],
                                    iterations=model_params['iterations'], 
                                    calculate_training_loss=model_params['calculate_training_loss'], 
                                    num_threads=model_params['num_threads'])
     # На вход item-user matrix
    model.fit(sparse_uim, show_progress=True)

    f_items = itemid_to_id[filter_item]
    res = u_ids.apply(lambda x: [id_to_itemid[id] for id in get_recommends(userid_to_id[x], 
                                                                           sparse_uim[userid_to_id[x]], 
                                                                           f_items,
                                                                           model=model,
                                                                           N=5)])
    return res

In [6]:
data = pd.read_csv('./input/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id', 'product_id': 'item_id'}, inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
filter_item = 999999
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = filter_item

e:\progi\python38\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.

Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные  λ  и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:

Значение параметра - время обучения  
Значение параметра - качество train, качество validation  
Сделайте качественные выводы  

In [9]:
model_params = {'factors': 100, 'regularization': 0.001, 'iterations': 15,  
                'calculate_training_loss': True, 'num_threads': 4}

In [10]:
result['q_sum_als'] = als(data_train, 'quantity', 'sum', result['user_id'], filter_item, model_params)
result['q_count_als'] = als(data_train, 'quantity', 'count', result['user_id'], filter_item, model_params)
result['q_mean_als'] = als(data_train, 'quantity', 'mean', result['user_id'], filter_item, model_params)

s = result.apply(lambda row: precision_at_k(row['q_sum_als'], row['actual']), axis=1).mean()
c = result.apply(lambda row: precision_at_k(row['q_count_als'], row['actual']), axis=1).mean()
m = result.apply(lambda row: precision_at_k(row['q_mean_als'], row['actual']), axis=1).mean()

e:\progi\python38\lib\site-packages\implicit\utils.py:26: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [11]:
s, c, m

(0.16395690499510285, 0.1803134182174339, 0.18903036238981394)

In [15]:
result.head(5)

,user_id,actual,q_sum_als,q_count_als,q_mean_als,best
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 979707, 965766, 1095275]","[1033142, 5569374, 979707, 995242, 9527290]","[995242, 1082185, 986947, 820165, 907014]","[820165, 986947, 866227, 1082185, 1095275]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 914190, 1055863, 9337581, 910032]","[929668, 1106523, 822407, 5568378, 1133018]","[1106523, 1133018, 938700, 910032, 844179]","[1055863, 938700, 1022003, 1133018, 910032]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[923746, 1051516, 878996, 866211, 1024306]","[1007195, 866211, 1023720, 1051516, 878996]","[878996, 1082185, 1024306, 1127831, 866211]","[1082185, 878996, 1024306, 1127831, 1007195]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[938700, 1082185, 1013321, 1133018, 1022003]","[1082185, 938700, 893018, 1013321, 1096036]","[1127831, 1013321, 1022003, 826249, 1030819]","[826249, 1082185, 1050229, 938700, 1030819]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1068719, 883404, 840361, 1053690, 965766]","[1065593, 6534178, 1004906, 1068719, 938700]","[1053690, 833715, 883404, 844165, 1004906]","[1068719, 938700, 1082185, 995242, 1053690]"


In [13]:
%%time
model_params = {'factors': 100, 'regularization': 0.07, 'iterations': 10,  
                'calculate_training_loss': True, 'num_threads': 15}
result['best'] = als(data_train, 'quantity', 'mean', result['user_id'], filter_item, model_params)
result.apply(lambda row: precision_at_k(row['best'], row['actual']), axis=1).mean()

  0%|          | 0/10 [00:00<?, ?it/s]

Wall time: 9.98 s


0.1952987267384917

### Вывод
Наилучший результат показала сводная по среднему количеству покупаемого пользователем товара.  
Наилучшим параметром с точки зрения оптимизации получаемого значения оказался regularization, а наилучшим с точки зрения повышения производительности num_threads.
